# PROCESADO DE DATOS DE CHIPS DE qPCR

El objetivo de este cuaderno es generar un script para abrir los distintos archivos de datos **agrupados previamente por muestra** y extraer de ellos la información pertinente 

In [1]:
import numpy as np
import pandas as pd
import os
import re
from pandas import DataFrame

Voy a hacer un loop para abrir uno a uno los archivos del directorio, por lo que no hay manera de ir mostrando los resultados de cada dataset poco a poco según filtro. Por ello iré generando un objeto que lleve la cuenta de la información relevante a cada paso a comparar: el "nº de ARG" (no del todo, pues en muchos casos hay 3 réplicas biológicas) que se tienen en cada paso de filtrado. De manera paralela, guardaré los datos de 16S sin filtrar para poder llevar a cabo los cálculos de presencia relativa y nº de copia e iré filtrando los datos según entren para conservar los ARGs. Véase:
1. Abro un archivo del directorio
2. Saco el nº de chip y lo añado como columna para no perder esa información. Aparte, evita comportamientos indeseados
3. Extraigo sin filtrar la información de 16S
4. Voy filtrando según criterios de bibliografía:
    1. Ha amplificado, tiene Ct
    2. Ct $\leq$ 27
    3. Eficiencia = [1.75; 2.2]
    4. Nº réplicas > 1

Finalmente, pasar los datos ya filtrados a tres .csv, para no tener que rehacer estos cálculos cada vez que quiera hacer nada (y poder meterles mano en excel, que siempre es más cómodo)

In [2]:
## Como voy a estar filtrando un df siempre en base a unos mismos criterios pero fragmentando previamente el df,
## va a resultar todo más limpio si creo una función para llevar a cabo el tipo de filtrado específico que quiero
## Hacer las cosas así también me permitirá una mayor flexibilidad para cambiar la manera de filtrar

# La función toma de entrada un df, así como el valor de Ct en el que cortar, el rango de eficiencia y el nº de réplicas.
# No hace falta definir cómo llevar a cabo el filtrado a nivel de tener amplificación porque ese filtrado es universal y no puede variar
# La "track_lst" es opcional pq no me interesa siempre (por ejemplo, para filtrar controles internos da igual)
# OJO: ES "MUTADORA"
def qpcr_filter(input_df:DataFrame, Ct_cutoff:int, eff_range:list, n_reps: int, track_lst: list = []) -> tuple[DataFrame, list]:
    input_df = input_df[(input_df["Ct"].notna())] # quitar aquellos que no han amplificado
    track_lst.append(input_df.shape[0]) # Seguimiento
    #print(track_lst)
    input_df["Ct"] = pd.to_numeric(input_df["Ct"], errors = "coerce") #se da el caso que los valores de Ct vienen registrados como strings
    input_df = input_df[(input_df["Ct"]) <= Ct_cutoff] # De "Ct_cutoff" para abajo
    track_lst.append(input_df.shape[0]) # Ídem
    #print(track_lst)
    input_df = input_df[(input_df["Efficiency"] >= eff_range[0]) & (input_df["Efficiency"] <= eff_range[1])] # En el rango de "eff_range"
    track_lst.append(input_df.shape[0]) # ídem
    #print(track_lst)
    input_df = input_df[(input_df["N"]) >= n_reps] # Por encima del nº de réplicas
    track_lst.append(input_df.shape[0]) # ídem
    #print(track_lst)
    return (input_df, track_lst)

In [3]:
# Primero, chequear que el cwd sea el que debe. Es ABSOLUTO, CHAPUCERO. NECESARIO CAMBIARLO
if os.getcwd() == "/home/juanma/OneDrive/p_resistoma_antártida/scripts": os.chdir("../data/raw_data")

# Esta variable contiene el nombre de todas las versiones distintas de 16S que he visto + 16S new 1, que no la
# he visto pero porsiaca existiera la añado
# La segunda, "control_names" hace lo mismo pero con nombres de controles internos
sixteen_names = ["16S new 2", "16S old 1", "16S old 2", "16S new 1"]
control_names = ["000NTC", "GENO"]

# Dos df vacíos en los que iré metiendo la información final (ya filtrada y con utilidad estadística)
filtered_data = pd.DataFrame()
all_sixteen = pd.DataFrame()
filtered_control_data = pd.DataFrame()

# Y un diccionario en el que meter la información de "seguimiento"
arg_per_step = {}

# Ahora, el loop
for chip_file in os.listdir():
    # Este primer bloque se encarga del trabajo previo
    arg_per_step_lst = [] # Iré metiendo la información del nº de args en cada paso de filtrado aquí
    fname = os.fsdecode(chip_file) 
    chip_num = re.search("\d", fname).group() # sacar el nº de chip
    current_data = pd.read_csv(fname, sep = ";", on_bad_lines = "warn")
    current_data["chip"] = chip_num # poner el nº de chip

    # Este bloque hace el trabajo relativo al 16S
    sixteen_data = current_data[current_data["Assay"].isin(sixteen_names)]
    all_sixteen = pd.concat([all_sixteen, sixteen_data]) # Los datos del 16S me los puedo llevar aquí pq los necesito tal cual, sin filtrar

    # Dividir la tabla en dos, según sea muestra o control interno
    current_data = current_data[~current_data["Assay"].isin(sixteen_names)]
    arg_data = current_data[~current_data["Sample"].isin(control_names)]
    arg_per_step_lst.append(arg_data.shape[0]) #el nº inicial: todos los arg
    control_data = current_data[current_data["Sample"].isin(control_names)]
    del current_data # Por limpieza
    
    # Y ya llevar a cabo los filtrados pertinentes
    arg_data, arg_per_step_lst = qpcr_filter(arg_data, 27, [1.75, 2.2], 2, arg_per_step_lst) # Correr la función. Es MUTADORA
    control_data = qpcr_filter(control_data, 27, [1.75, 2.2], 2)[0] # Lo mismo para el control, pero manteniendo solo el df
    
    # Por último, ir guardando la información en su objeto definitivo
    arg_per_step[chip_num] = arg_per_step_lst # meter los datos en el diccionario
    filtered_data = pd.concat([filtered_data, arg_data]) # idem en un df
    filtered_control_data = pd.concat([filtered_control_data, control_data])
    #print(control_data)
    
control_data
#print(arg_per_step)

/tmp/ipykernel_14131/1082640017.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df["Ct"] = pd.to_numeric(input_df["Ct"], errors = "coerce") #se da el caso que los valores de Ct vienen registrados como strings
/tmp/ipykernel_14131/1082640017.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df["Ct"] = pd.to_numeric(input_df["Ct"], errors = "coerce") #se da el caso que los valores de Ct vienen registrados como strings
/tmp/ipykernel_14131/1082640017.py:13: SettingWithCopyWarning: 
A value 

,Sample,Assay,N,Outliers,Rejected,Total reps,Ct,Ct SD,Tm,Tm SD,Efficiency,Conc,Flags,chip
949,GENO,fabK,2,0,0,2,26.55,0.52,77.86,0.05,1.82,-1,NaN,6
1419,GENO,tetPA,2,0,0,2,24.60,0.20,75.13,0.06,1.88,-1,NaN,6
1544,GENO,czcA,2,0,0,2,21.25,0.18,78.12,0.05,1.87,-1,NaN,6


In [4]:
# Antes de seguir, me interesa convertir el diccionario en un df para que sea más claro interpretarlo.

step_explanation = ["start", "amplified", "under_27", "eff_range", "2_reps"] # Dar nombre a qué está sucediendo en cada filtrado

loss_stats = pd.DataFrame.from_dict(arg_per_step, orient = "index", columns = step_explanation)
loss_stats.sort_index(inplace = True) # Por razones que no me da tiempo a entender, python abre los .csv del directorio cada vez de manera distinta

loss_stats

,start,amplified,under_27,eff_range,2_reps
1,1143,534,39,35,25
2,1143,555,23,22,20
3,1143,580,55,53,49
4,1143,682,79,72,63
5,1143,504,27,27,25
6,1143,610,87,86,73
7,1143,778,708,664,499
8,1143,1052,1005,957,878
9,1143,1019,959,897,824


In [5]:
# Y ya solo quedaría pasar todo a .csv

filtered_data.to_csv("/home/juanma/OneDrive/p_resistoma_antártida/data/clean_data/filtered_arg")
loss_stats.to_csv("/home/juanma/OneDrive/p_resistoma_antártida/data/clean_data/loss_stats")
all_sixteen.to_csv("/home/juanma/OneDrive/p_resistoma_antártida/data/clean_data/16S_refs")
control_data.to_csv("/home/juanma/OneDrive/p_resistoma_antártida/data/clean_data/filtered_controls")

Idealmente, sabría qué significa "GENO", pues tenemos la mala suerte de que lo único que he encontrado mencionado en la bibliografía (000NTC) no cumple las condiciones de filtrado en ningún chip. Así que, para curarme de espanto, hago dos filtrados diferentes: uno sin quitar GENO y otro quitando aquellos positivos que tengan réplica en GENO. 

Así, tal cual: <mark>si GENO cumple los criterios de filtrado</mark> (véase, si llegados a este punto sigue presente en el df "filtered_data"), <mark>aquellos ARGs en los que lo haga para los chips en los que lo haga, fuera.</mark>

**ESTO ES UNA SUPOSICIÓN Y LO CORRECTO SERÍA PREGUNTAR**

In [6]:
control_data

,Sample,Assay,N,Outliers,Rejected,Total reps,Ct,Ct SD,Tm,Tm SD,Efficiency,Conc,Flags,chip
949,GENO,fabK,2,0,0,2,26.55,0.52,77.86,0.05,1.82,-1,NaN,6
1419,GENO,tetPA,2,0,0,2,24.60,0.20,75.13,0.06,1.88,-1,NaN,6
1544,GENO,czcA,2,0,0,2,21.25,0.18,78.12,0.05,1.87,-1,NaN,6
